# Survival Prediction for Titanic Dataset

**This notebook is to predict survival classes from the famous titanic dataset. It presents exploratory data analysis and visualization as well as predictive modeling with different classification algorithms including neural network. Finally, it also shows how to build a simple yet powerful pipeline.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import warnings
warnings.filterwarnings('ignore')

**Loading the dataset**

In [ ]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')     

# **Exploratory Data Analysis and Visualization**

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape, test.shape)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def bar_chart(feature):
    survived=train[train['Survived']==1][feature].value_counts()
    dead=train[train['Survived']==0][feature].value_counts()
    df=pd.DataFrame([survived,dead])
    df.index=['survived','dead']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

In [ ]:
bar_chart('Sex')

In [ ]:
bar_chart('Pclass')

In [ ]:
bar_chart('Embarked')

In [ ]:
bar_chart('SibSp')

In [ ]:
bar_chart('Parch')

**Feature Engineering**

In [ ]:
train['Age'].fillna(train['Age'].mean(),inplace=True)
test['Age'].fillna(test['Age'].mean(),inplace=True)
test['Fare'].fillna(test['Fare'].mean(),inplace=True)
train['Embarked'].fillna(value='S',inplace=True)

In [ ]:
train['family']=train['SibSp']+train['Parch']+1
test['family']=test['SibSp']+train['Parch']+1

In [ ]:
train['Sex'] = train['Sex'].replace(['female','male'],[0,1])
train['Embarked'] = train['Embarked'].replace(['S','Q','C'],[1,2,3])

In [ ]:
test['Sex'] = test['Sex'].replace(['female','male'],[0,1])
test['Embarked'] = test['Embarked'].replace(['S','Q','C'],[1,2,3])

In [ ]:
train_clean=train.drop(columns=['PassengerId','Name','SibSp','Parch','Ticket','Cabin'])
test_clean=test.drop(columns=['PassengerId','Name','SibSp','Parch','Ticket','Cabin'])

In [ ]:
train_clean

# Predictive Modeling

**Logistic Regression**

In [ ]:
X_train=train_clean.drop(columns=['Survived'])
y_train=train_clean[['Survived']]

In [ ]:
from sklearn.preprocessing import StandardScaler
X_train_scale=StandardScaler().fit_transform(X_train)
pd.DataFrame(X_train_scale).head()

In [ ]:
from sklearn.linear_model import LogisticRegression

LR=LogisticRegression().fit(X_train_scale, y_train)
y_pred=LR.predict(test_clean)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_pred, gender_submission['Survived']))

In [ ]:
from sklearn.model_selection import cross_val_score

scores=cross_val_score(LogisticRegression(),X_train_scale,y_train,cv=5)
print(scores)
print(scores.mean()) # generalization score

**Grid Search CV**

`Hyperparameter Tuning`

In [ ]:
from sklearn.model_selection import GridSearchCV

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]} # C = 1/alpha
grid

In [ ]:
score=GridSearchCV(LogisticRegression(),grid).fit(X_train_scale, y_train)

print(score.best_params_)
print(score.best_score_)

**Submission File Preparation**

In [ ]:
data = {'PassengerId':gender_submission['PassengerId'],
        'Survived':y_pred}
result=pd.DataFrame(data)
result.to_csv('/kaggle/working/result_lr.csv', index=False)

**Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier().fit(X_train_scale, y_train)
y_pred_rf=RF.predict(test_clean)
from sklearn.metrics import classification_report
#print(classification_report(y_pred_rf, gender_submission['Survived']))
#print(y_pred_rf)
scores=cross_val_score(RandomForestClassifier(), X_train_scale, y_train, cv=5)
print(scores)
print(scores.mean())

In [ ]:
data={'PassengerId': gender_submission['PassengerId'],'Survived':y_pred_rf}
result_rf=pd.DataFrame(data)
result_rf.to_csv('/kaggle/working/result_rf.csv', index=False)
# result_rf1=pd.read_csv('/kaggle/working/result_rf.csv')

**Support Vector Classifier**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
svc=SVC(kernel='linear', C=1)
scores=cross_val_score(svc, X_train_scale, y_train, cv=5)
print(scores)
print(scores.mean())

In [ ]:
y_pred_svc=SVC(kernel='linear', C=1).fit(X_train_scale, y_train).predict(test_clean)
data={'PassengerId': gender_submission['PassengerId'],'Survived':y_pred_svc}
result_svc=pd.DataFrame(data)
result_svc.to_csv('/kaggle/working/result_svc.csv', index=False)
result_svc=pd.read_csv('/kaggle/working/result_svc.csv')

**Decision Tree Classifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
dtc=DecisionTreeClassifier()
scores=cross_val_score(dtc, X_train, y_train, cv=5)
print(scores)
print(scores.mean())

y_pred_dtc=DecisionTreeClassifier().fit(X_train, y_train).predict(test_clean)
data={'PassengerId': gender_submission['PassengerId'],'Survived':y_pred_dtc}
result_dtc=pd.DataFrame(data)
result_dtc.to_csv('/kaggle/working/result_dtc.csv', index=False)
result_dtc=pd.read_csv('/kaggle/working/result_dtc.csv')

**Naive Bayes Classifier**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
nb=GaussianNB()
scores=cross_val_score(nb, X_train, y_train, cv=5)
print(scores)
print(scores.mean())

y_pred_nb=GaussianNB().fit(X_train, y_train).predict(test_clean)
data={'PassengerId': gender_submission['PassengerId'],'Survived':y_pred_nb}
result_nb=pd.DataFrame(data)
result_nb.to_csv('/kaggle/working/result_nb.csv', index=False)
result_nb=pd.read_csv('/kaggle/working/result_nb.csv')